In [1]:

import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from utils import iouUtils, calcIOU, lrScheduler
from tensorflow.keras.callbacks import ModelCheckpoint
import sys, os
from keras.regularizers import l2
from YOLOv1_learning_Rate import customLearningRate
from YOLOv1_Reshape_Layer import YOLOv1_LastLayer_Reshape
from YOLOv1_Loss import YOLOv1_loss
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

here = os.path.dirname(".")
sys.path.append(os.path.join(here, '..'))

from dataHandler import *

2024-07-24 15:53:31.022778: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 15:53:31.036969: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 15:53:31.041169: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 15:53:31.051839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 15:53:31.857218: W tensorflow/compiler/tf2

Num GPUs Available:  1


I0000 00:00:1721823812.850687   54971 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721823812.880301   54971 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721823812.880474   54971 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721823812.881328   54971 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [2]:
# YOLOv1 structure
YOLOv1_inputShape = (448,448,3) # Shape of the input image 
classNo = 1 # Number of classes we are trying to detect
input = tf.keras.layers.Input(shape=YOLOv1_inputShape)
leakyReLu = tf.keras.layers.LeakyReLU(negative_slope = .1)


# The backbone, Acts ads a feature extractor
# L1
x = tf.keras.layers.Conv2D(filters = 64, kernel_size=7, strides = 2, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(input)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L2
x = tf.keras.layers.Conv2D(filters = 192, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L3
x = tf.keras.layers.Conv2D(filters = 128, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 256, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 256, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L4
for _ in range(4):
    x = tf.keras.layers.Conv2D(filters = 256, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
    x = tf.keras.layers.Conv2D(filters = 512, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding = "same")(x)

# L5
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 512, kernel_size=1, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 2, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)

# L6
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)
x = tf.keras.layers.Conv2D(filters = 1024, kernel_size=3, strides = 1, padding = "same", activation= leakyReLu, kernel_regularizer=l2(1e-5))(x)

# Neck
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(4096)(x)
x = tf.keras.layers.Dense(7*7*(5*2+classNo), activation="sigmoid")(x)
x = tf.keras.layers.Dropout(.5)(x) # Dropout layer for avoiding overfitting
x = YOLOv1_LastLayer_Reshape((7,7,5*2+classNo))(x)
model = tf.keras.Model(inputs = input, outputs = x, name = "YOLOv1")

model.compile(loss = YOLOv1_loss ,optimizer = 'adam')

In [3]:
# See if the directory to save the checkpoints exists
if not os.path.isdir(f"{os.getcwd()}/model_data"):
    os.mkdir(f"{os.getcwd()}/model_data")

# Instantiate the checkpoint object
chkPoint = ModelCheckpoint(filepath='./model_data/model_{epoch:02d}-{val_loss:.2f}.keras',
                                    save_best_only=True,
                                    monitor='val_loss',
                                    mode='min',
                                    verbose=1
                            )

batch_size = 1
LR_schedule = [
    (0, 0.01),
    (75, 0.001),
    (105, 0.0001),
]

dfTrain = annotationsToDataframe(f"../data/labels/train", "txt")
trainingBatchGenerator = dataGenerator_YOLOv1(f"../data/images/train", batch_size, (448,448), dfTrain, 1, True)

dfTest = annotationsToDataframe(f"../data/labels/test", "txt")
testingBatchGenerator = dataGenerator_YOLOv1(f"../data/images/test", batch_size, (448,448), dfTrain, 1, True)

model.fit(x=trainingBatchGenerator,
        # steps_per_epoch = int(dfTrain.shape[0] // batch_size),
        epochs = 135,
        verbose = 1,
        validation_data = testingBatchGenerator,
        # validation_steps = int(len(dfTest) // batch_size),
        callbacks = [
            customLearningRate(lrScheduler, LR_schedule),
            chkPoint,
        ]
)

4105
Updated the learning rate at epoch NO. 0. New learning rate: 0.01
Epoch 1/135


I0000 00:00:1721823819.048758   55037 service.cc:146] XLA service 0x79836c0022c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721823819.048785   55037 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-07-24 15:53:39.284266: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-24 15:53:39.898972: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-07-24 15:53:41.835095: W external/local_tsl/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-07-24 15:53:41.835132: W external/local_tsl/tsl/framework/bfc_allocator.cc:291]

  1/982 ━━━━━━━━━━━━━━━━━━━━ 4:08:59 15s/step - loss: 17.8916

2024-07-24 15:53:48.445029: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_add_subtract_fusion_12', 20 bytes spill stores, 20 bytes spill loads

I0000 00:00:1721823828.477638   55037 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


126/982 ━━━━━━━━━━━━━━━━━━━━ 1:15 89ms/step - loss: nan

KeyboardInterrupt: 